
# 時間の経過によるバックプロパゲーション

:ラベル: `sec_bptt`

 :numref: `sec_rnn-scratch`の演習を完了した場合は、時折発生する大きな勾配によってトレーニングが不安定になるのを防ぐために、勾配クリッピングが不可欠であることがわかるでしょう。私たちは、爆発的な勾配が長いシーケンスにわたる逆伝播に起因することをほのめかしました。多数の最新の RNN アーキテクチャを紹介する前に、シーケンス モデルで*バックプロパゲーションが*どのように機能するかを数学的に詳しく見てみましょう。この議論により、勾配*の消失*と*爆発*の概念がある程度正確になることを願っています。 :numref: `sec_backprop`で MLP を導入したときに、計算グラフを介した順方向伝播と逆方向伝播について説明したことを思い出していただければ、RNN における順方向伝播は比較的簡単であるはずです。 RNN にバックプロパゲーションを適用することは、*時間によるバックプロパゲーション*と呼ばれます:cite: `Werbos.1990` 。この手順では、RNN の計算グラフを一度に 1 タイム ステップずつ展開 (展開) する必要があります。アンロールされた RNN は本質的に、同じパラメーターがアンロールされたネットワーク全体で繰り返され、各タイム ステップで現れるという特別な特性を持つフィードフォワード ニューラル ネットワークです。次に、他のフィードフォワード ニューラル ネットワークと同様に、展開されたネットを通じて勾配を逆伝播する連鎖規則を適用できます。各パラメータに関する勾配は、展開されたネット内でパラメータが発生するすべての場所にわたって合計する必要があります。このような重み付けの処理については、畳み込みニューラル ネットワークに関する章でよく知られているはずです。

シーケンスがかなり長くなる可能性があるため、複雑な問題が発生します。 1,000 を超えるトークンで構成されるテキスト シーケンスを扱うことは珍しいことではありません。これは、計算 (メモリが多すぎる) と最適化 (数値の不安定性) の両方の観点から問題を引き起こすことに注意してください。最初のステップからの入力は、出力に到達する前に 1000 を超える行列積を通過し、勾配を計算するにはさらに 1000 の行列積が必要です。ここでは、何が問題になる可能性があるのか​​、そして実際にどのように対処すればよいのかを分析します。

##  RNN における勾配の解析

:label: `subsec_bptt_analysis`

 RNN がどのように動作するかを示す単純化されたモデルから始めます。このモデルは、非表示状態の詳細とそれがどのように更新されるかについての詳細を無視します。ここでの数学的表記法では、スカラー、ベクトル、行列を明示的に区別しません。私たちは直感を養おうとしているだけです。この単純化されたモデルでは、タイム ステップ $t$ での $h_t$ を隠れ状態、$x_t$ を入力、$o_t$ を出力として表します。 :numref: `subsec_rnn_w_hidden_states`での議論を思い出してください。入力と隠れ状態は、隠れ層の 1 つの重み変数で乗算される前に連結できるということです。したがって、$w_h$ と $w_o$ を使用して、それぞれ隠れ層と出力層の重みを示します。その結果、各タイム ステップでの隠れ状態と出力は次のようになります。

 $$\begin{aligned}h_t &amp;= f(x_t, h_{t-1}, w_h),\o_t &amp;= g(h_t, w_o),\end{aligned}$$ :eqlabel: `eq_bptt_ht_ot`

ここで、$f$ と $g$ はそれぞれ隠れ層と出力層の変換です。したがって、値のチェーン ${\ldots, (x_{t-1}, h_{t-1}, o_{t-1}), (x_{t}, h_{t}, o_t), \ldots}$ は再帰計算によって相互に依存します。順伝播は非常に簡単です。必要なのは、$(x_t, h_t, o_t)$ トリプルを一度に 1 タイム ステップずつループすることだけです。出力 $o_t$ と目的のターゲット $y_t$ の間の不一致は、すべての $T$ タイム ステップにわたる目的関数によって次のように評価されます。

 $$L(x_1, \ldots, x_T, y_1, \ldots, y_T, w_h, w_o) = \frac{1}{T}\sum_{t=1}^T l(y_t, o_t).$$

バックプロパゲーションの場合、特に目的関数 $L$ のパラメーター $w_h$ に関する勾配を計算する場合、問題は少し複雑になります。具体的に言うと、連鎖の法則により、

 $$\begin{aligned}\frac{\partial L}{\partial w_h} &amp; = \frac{1}{T}\sum_{t=1}^T \frac{\partial l(y_t, o_t)} {\partial w_h} \&amp; = \frac{1}{T}\sum_{t=1}^T \frac{\partial l(y_t, o_t)}{\partial o_t} \frac{\partial g(h_t , w_o)}{\partial h_t} \frac{\partial h_t}{\partial w_h}.\end{aligned}$$ :eqlabel: `eq_bptt_partial_L_wh`

 :eqref: `eq_bptt_partial_L_wh`の積の 1 番目と 2 番目の因子は計算が簡単です。 3 番目の要素 $\partial h_t/\partial w_h$ は、$h_t$ に対するパラメーター $w_h$ の効果を繰り返し計算する必要があるため、処理が難しくなります。 :eqref: `eq_bptt_ht_ot`再帰計算によると、 $h_t$ は $h_{t-1}$ と $w_h$ の両方に依存し、 $h_{t-1}$ の計算も $w_h$ に依存します。したがって、連鎖規則を使用して $w_h$ に関する $h_t$ の導関数の合計を評価すると、次の結果が得られます。

 $$\frac{\partial h_t}{\partial w_h}= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h} +\frac{\partial f( x_{t},h_{t-1},w_h)}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_h}.$$ :eqlabel: `eq_bptt_partial_ht_wh_recur`

上記の勾配を導出するには、$a_{0}=0$ および $a_{t}= を満たす 3 つのシーケンス ${a_{t}}、{b_{t}}、{c_{t}}$ があると仮定します。 b_{t}+c_{t}a_{t-1}$ ($t=1, 2,\ldots$)。 $t\geq 1$ の場合は、簡単に表示できます。

 $$a_{t}=b_{t}+\sum_{i=1}^{t-1}\left(\prod_{j=i+1}^{t}c_{j}\right)b_{ i}.$$ :eqlabel: `eq_bptt_at`

 $a_t$、$b_t$、$c_t$ を次のように置き換えることにより、

 $$\begin{aligned}a_t &amp;= \frac{\partial h_t}{\partial w_h},\ b_t &amp;= \frac{\partial f(x_{t},h_{t-1},w_h)}{ \partial w_h}, \ c_t &amp;= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}},\end{aligned}$$

 :eqref: `eq_bptt_partial_ht_wh_recur`の勾配計算は $a_{t}=b_{t}+c_{t}a_{t-1}$ を満たします。したがって、 :eqref: `eq_bptt_at`ごとに、 :eqref: `eq_bptt_partial_ht_wh_recur`の反復計算を削除できます。

 $$\frac{\partial h_t}{\partial w_h}=\frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h}+\sum_{i=1} ^{t-1}\left(\prod_{j=i+1}^{t} \frac{\partial f(x_{j},h_{j-1},w_h)}{\partial h_{j -1}} \right) \frac{\partial f(x_{i},h_{i-1},w_h)}{\partial w_h}.$$ :eqlabel: `eq_bptt_partial_ht_wh_gen`

連鎖ルールを使用して $\partial h_t/\partial w_h$ を再帰的に計算できますが、$t$ が大きい場合は常にこの連鎖が非常に長くなる可能性があります。この問題に対処するためのいくつかの戦略について説明しましょう。

### 完全な計算

1 つのアイデアは、 :eqref: `eq_bptt_partial_ht_wh_gen`で合計を計算することです。ただし、初期条件の微妙な変化が結果に大きな影響を与える可能性があるため、これは非常に遅く、勾配が爆発する可能性があります。つまり、初期条件の最小限の変化が結果の不均衡な変化につながるバタフライ効果に似た現象が見られる可能性があります。これは一般に望ましくないことです。結局のところ、私たちはよく一般化できる堅牢な推定量を探しています。したがって、この戦略は実際にはほとんど使用されません。

### タイムステップの切り詰め###

あるいは、$\tau$ ステップの後で :eqref: `eq_bptt_partial_ht_wh_gen`の合計を切り捨てることもできます。これが私たちがこれまで議論してきたことです。これにより、$\partial h_{t-\tau}/\partial w_h$ で合計を終了するだけで、真の勾配の*近似*が得られます。実際には、これは非常にうまく機能します。これは、一般に時間経過による切り捨て逆伝播と呼ばれるものです (cite: `Jaeger.2002` )。この結果の 1 つは、モデルが長期的な結果ではなく、主に短期的な影響に焦点を当てていることです。これは、推定がより単純でより安定したモデルに偏るため、実際には*望ましいこと*です。

### ランダム化された切り捨て

最後に、$\partial h_t/\partial w_h$ を確率変数に置き換えることができます。これは期待どおりですが、シーケンスを切り詰めます。これは、$P(\xi_t = 0) = 1-\pi_t$ および $P(\xi_t = \pi_t^{ -1}) = \pi_t$、したがって $E[\xi_t] = 1$。これを使用して、:eqref: `eq_bptt_partial_ht_wh_recur`の勾配 $\partial h_t/\partial w_h$ を次のように置き換えます。

 $$z_t= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h} +\xi_t \frac{\partial f(x_{t},h_{t- 1},w_h)}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_h}.$$

 $\xi_t$ の定義から、$E[z_t] = \partial h_t/\partial w_h$ となります。 $\xi_t = 0$ の場合、再帰計算はそのタイム ステップ $t$ で終了します。これにより、さまざまな長さのシーケンスの加重和が得られます。長いシーケンスはまれですが、適切に過重に重み付けされます。このアイデアは :citet: `Tallec.Ollivier.2017`によって提案されました。

### 戦略の比較

![](../img/truncated-bptt.svg) :label: `fig_truncated_bptt`

 :numref: `fig_truncated_bptt` 、RNN の時間に対する逆伝播を使用して*Time Machine*の最初の数文字を分析するときの 3 つの戦略を示しています。
- 最初の行は、テキストをさまざまな長さのセグメントに分割するランダム化された切り詰めです。
-  2 行目は、テキストを同じ長さのサブシーケンスに分割する通常の切り捨てです。これは私たちが RNN 実験で行ってきたことです。
-  3 行目は、計算上実行不可能な式につながる時間にわたる完全な逆伝播です。

残念ながら、ランダム化された切り捨ては、理論的には魅力的ですが、おそらく多くの要因により、通常の切り捨てよりもうまく機能しません。まず、過去へのバックプロパゲーション ステップを何度も行った後の観察の効果は、実際に依存関係を捉えるのに十分です。第 2 に、分散の増加により、ステップが増えるほど勾配がより正確になるという事実が相殺されます。第三に、実際には、短い範囲の相互作用しか持たないモデルが*必要です*。したがって、時間の経過とともに規則的に切り詰められたバックプロパゲーションには、望ましい可能性があるわずかな規則化効果があります。

## 経時的な逆伝播の詳細

一般原理を説明した後、時間によるバックプロパゲーションについて詳しく説明します。 :numref: `subsec_bptt_analysis`の分析とは異なり、以下では、すべての分解されたモデル パラメーターに関して目的関数の勾配を計算する方法を示します。物事を単純にするために、バイアス パラメーターのない RNN を考えます。隠れ層の活性化関数は恒等マッピング ($\phi(x)=x$) を使用します。タイムステップ $t$ の場合、単一の入力例とターゲットをそれぞれ $\mathbf{x}_t \in \mathbb{R}^d$ と $y_t$ とします。隠れ状態 $\mathbf{h}_t \in \mathbb{R}^h$ と出力 $\mathbf{o}_t \in \mathbb{R}^q$ は次のように計算されます。

 $$\begin{aligned}\mathbf{h} *t &amp;= \mathbf{W}* {hx} \mathbf{x} *t + \mathbf{W}* {hh} \mathbf{h} *{t-1},\ \mathbf{o}_t &amp;= \mathbf{W}* {qh} \mathbf{h}_{t},\end{aligned}$$

ここで $\mathbf{W} *{hx} \in \mathbb{R}^{h \times d}$、$\mathbf{W}* {hh} \in \mathbb{R}^{h \times h}$ 、 $\mathbf{W}_{qh} \in \mathbb{R}^{q \times h}$ は重みパラメータです。タイムステップ $t$ での損失を $l(\mathbf{o}_t, y_t)$ で表します。目的関数であるシーケンスの開始から $T$ タイム ステップにわたる損失は次のようになります。

 $$L = \frac{1}{T} \sum_{t=1}^T l(\mathbf{o}_t, y_t).$$

 RNN の計算中のモデル変数とパラメーター間の依存関係を視覚化するために、 :numref: `fig_rnn_bptt`に示すように、モデルの計算グラフを描画できます。たとえば、タイム ステップ 3 の隠れ状態の計算 $\mathbf{h} *3$ は、モデル パラメーター $\mathbf{W} {hx}$ および $\mathbf{W}_{hh}$ に依存します。* 、最後のタイムステップ $\mathbf{h}_2$ の非表示状態、および現在のタイムステップ $\mathbf{x}_3$ の入力。 

![](../img/rnn-bptt.svg) :ラベル: `fig_rnn_bptt`

先ほど述べたように、:numref: `fig_rnn_bptt`のモデル パラメーターは、$\mathbf{W} *{hx}$、$\mathbf{W}* {hh}$、および $\mathbf{W} *{qh}$ です。*一般に、このモデルをトレーニングするには、$\partial L/\partial \mathbf{W} {hx}$、$\partial L/\partial \mathbf{W} *{hh}$、および $\**のパラメーターに関する勾配計算が必要です。**部分的な L/\部分的な \mathbf{W}* {qh}$。 :numref: `fig_rnn_bptt`の依存関係に従って、矢印の反対方向に移動して、勾配を順番に計算して保存できます。連鎖ルールでさまざまな形状の行列、ベクトル、スカラーの乗算を柔軟に表現するために、 :numref: `sec_backprop`で説明されている $\text{prod}$ 演算子を引き続き使用します。

まず第一に、任意のタイム ステップ $t$ におけるモデル出力に関して目的関数を微分することは非常に簡単です。

 $$\frac{\partial L}{\partial \mathbf{o}_t} = \frac{\partial l (\mathbf{o}_t, y_t)}{T \cdot \partial \mathbf{o}_t} \in \mathbb{R}^q.$$ :eqlabel: `eq_bptt_partial_L_ot`

これで、*出力層のパラメーター $\mathbf{W} {qh}$ に関する目的関数の勾配を計算できます。 $\partial L/\partial \mathbf{W}* {qh} \in \mathbb{ R}^{q \times h}$。 :numref: `fig_rnn_bptt`に基づいて、目的 $L$ は $\mathbf{o}_1、\ldots、\mathbf{o}_T$ を介して $\mathbf{W}_{qh}$ に依存します。連鎖規則を使用すると、次の結果が得られます

$$ \frac{\partial L}{\partial \mathbf{W} *{qh}} = \sum* {t=1}^T \text{prod}\left(\frac{\partial L}{\partial \ mathbf{o} *t}, \frac{\partial \mathbf{o}_t}{\partial \mathbf{W}* {qh}}\right) = \sum_{t=1}^T \frac{\partial L }{\partial \mathbf{o}_t} \mathbf{h}_t^\top, $$

ここで $\partial L/\partial \mathbf{o}_t$ は :eqref: `eq_bptt_partial_L_ot`で与えられます。

次に、 :numref: `fig_rnn_bptt`に示すように、最終タイム ステップ $T$ では、目的関数 $L$ は $\mathbf{o}_T$ を介してのみ隠れ状態 $\mathbf{h}_T$ に依存します。したがって、連鎖則を使用して、勾配 $\partial L/\partial \mathbf{h}_T \in \mathbb{R}^h$ を簡単に見つけることができます。

 $$\frac{\partial L}{\partial \mathbf{ *h} T} = \text{prod}\left(\frac{\partial L}{\partial \mathbf{o}_T}, \frac{\部分 \mathbf{o}_T}{\部分 \mathbf{h}_T} \right) = \mathbf{W} {qh}^\top \frac{* \部分 L}{\部分 \mathbf{o}_T} .$$ :eqlabel: `eq_bptt_partial_L_hT_final_step`

タイムステップ $t &lt; T$ ではさらに複雑になります。目的関数 $L$ は*、$\mathbf{h} {t+1}$ および $\mathbf{o} を介して*$\mathbf{h} t$ に依存します。 _t$。連鎖則によれば、任意のタイム ステップ $t &lt; T$ における隠れ状態 $\partial L/\partial \mathbf{h}_t \in \mathbb{R}^h$ の勾配は次のように再帰的に計算できます。

 $$\frac{\partial L}{\partial \mathbf{h} *t} = \text{prod}\left(\frac{\partial L}{\partial \mathbf{h}* {t+1}}, *\frac{\partial \mathbf{h} {t+1}}{\partial \mathbf{h}_t} \right) + \text{prod}\left(\frac{\partial L}{\partial \mathbf {o}_t}, \frac{\partial \mathbf{o}_t}{\partial \mathbf{h}_t} \right) = \mathbf{W} {hh}^\top \frac{\partial L}* {\partial \mathbf{h} *{t+1}} + \mathbf{W}* {qh}^\top \frac{\partial L}{\partial \mathbf{o}_t}.$$ :eqlabel: `eq_bptt_partial_L_ht_recur`

解析のために、任意のタイム ステップ $1 \leq t \leq T$ の反復計算を拡張すると、次のようになります。

 $$\frac{\partial L}{\partial \mathbf{h} *t}= \sum* {i=t}^T {\left(\mathbf{W} *{hh}^\top\right)}^{ Ti} \mathbf{W}* {qh}^\top \frac{\partial L}{\partial \mathbf{o}_{T+ti}}.$$ :eqlabel: `eq_bptt_partial_L_ht`

 :eqref: `eq_bptt_partial_L_ht`から、この単純な線形例が長いシーケンス モデルのいくつかの重要な問題をすでに示していることがわかります。これには、潜在的に $\mathbf{W}_{hh}^\top$ の非常に大きな累乗が含まれます。その中で、1 より小さい固有値は消滅し、1 より大きい固有値は発散します。これは数値的に不安定であり、勾配の消失や爆発という形で現れます。これに対処する 1 つの方法は、 :numref: `subsec_bptt_analysis`で説明されているように、計算上便利なサイズでタイム ステップを切り捨てることです。実際には、この切り捨ては、指定されたタイム ステップ数の後に勾配を切り離すことによっても行うことができます。後で、長期短期記憶などのより洗練された順序モデルがこれをさらに軽減できることを見ていきます。

最後に、:numref: `fig_rnn_bptt` 、目的関数 $L$ が隠れ状態 $\mathbf{ を介して隠れ層のモデル パラメーター $\mathbf{W} *{hx}$ と $\mathbf{W}* {hh}$ に依存することを示しています。 h} *1、\ldots、\mathbf{h}_T$。このようなパラメータに関する勾配を計算するには $\partial L / \partial \mathbf{W}* {hx} \in \mathbb{R}^{h \times d}$ および $\partial L / \partial \mathbf{W }_{hh} \in \mathbb{R}^{h \times h}$ では、次のような連鎖規則を適用します。

 $$ \begin{aligned} \frac{\partial L}{\partial \mathbf{W} *{hx}} &amp;= \sum* {t=1}^T \text{prod}\left(\frac{\partial L}{\partial \mathbf{h} *t}, \frac{\partial \mathbf{h}_t}{\partial \mathbf{W} {* hx}}\right) = \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{h *} t} \mathbf{x}_t^\top,\ \frac{\partial L}{\partial \mathbf{W} {* hh}} &amp;= \ sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \mathbf{h} *t}, \frac{\partial \mathbf{h}_t}{\partial \ mathbf{W}* {hh}}\right) = \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{h} *t} \mathbf{h}* {t-1}^\上、\end{整列} $$

ここで、:eqref: `eq_bptt_partial_L_hT_final_step`および :eqref: `eq_bptt_partial_L_ht_recur`によって再帰的に計算される $\partial L/\partial \mathbf{h}_t$ は、数値安定性に影響を与える重要な量です。

 :numref: `sec_backprop`で説明したように、時間によるバックプロパゲーションは RNN におけるバックプロパゲーションの応用であるため、トレーニング RNN は順方向伝播と時間によるバックプロパゲーションを交互に実行します。さらに、時間を経たバックプロパゲーションにより、上記の勾配が順番に計算され、保存されます。具体的には、$\partial L / \partial \mathbf{h *} t$ を $\partial L / \partial \mathbf{W} {hx の両方の計算に使用するために*保存するなど、重複した計算を避けるために保存された中間値が再利用されます。 }$ と $\partial L / \partial \mathbf{W}_{hh}$ です。

## まとめ

時間によるバックプロパゲーションは、隠れ状態を持つシーケンス モデルへのバックプロパゲーションの単なる適用です。定期的な切り捨てやランダム化された切り捨てなど、計算の利便性と数値の安定性のために切り捨てが必要です。行列のべき乗が大きいと、固有値が発散したり、固有値が消滅したりする可能性があります。これは、勾配の爆発または消滅という形で現れます。計算を効率的に行うために、中間値は時間の経過とともに逆伝播中にキャッシュされます。

## 演習
1. 固有値 $\lambda_i$ を持つ対称行列 $\mathbf{M} \in \mathbb{R}^{n \times n}$ があり、その対応する固有ベクトルは $\mathbf{v} *i$ ($i = 1、\ldots、n$)。一般性を失わずに、$|\lambda_i| の順序で順序付けされていると仮定します。 \geq |\lambda* {i+1}|$。<ol><li> $\mathbf{M}^k$ に固有値 $\lambda_i^k$ があることを示します。
1. ランダムなベクトル $\mathbf{x} \in \mathbb{R}^n$ の場合、高い確率で $\mathbf{M}^k \mathbf{x}$ が固有ベクトル $\ とほぼ一致することを証明してください。 $\mathbf{M}$ の mathbf{v}_1$。この声明を正式なものにしてください。
1. 上記の結果は RNN の勾配にとって何を意味しますか?

[ディスカッション](https://discuss.d2l.ai/t/334)
